# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\Victor Lopez\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

price_data_path = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(price_data_path, "**","*.parquet"), recursive=True)

print(f"There are {len(parquet_files)} files") # Number of parquet files found
print(parquet_files[:3])  # Show the first 3 files


There are 2837 files
['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Read all parquet files into a single Dask DataFrame
ddf = dd.read_parquet(parquet_files)
#ddf.head()
#print(ddf.columns) #verify column names

#function to generate new features for each ticker group
def add_features(df):
    df = df.sort_values("Date") #sort data by date first to ensure lag and rolling calculations 
    df["Close_lag_1"] = df["Close"].shift(1)
    df["Adj_Close_lag_1"] = df["Adj Close"].shift(1)
    df["Returns"] = (df["Close"] / df["Close_lag_1"]) - 1
    df["Hi_lo_range"] = df["High"] - df["Low"]
    return df

# Apply the feature engineering function to each ticker group within each partition
dd_feat = ddf.map_partitions(lambda df: df.groupby("ticker").apply(add_features))

# Compute and preview the first 10 rows of the final dataset
dd_feat.compute().head(10)

C:\Users\Victor Lopez\AppData\Local\Temp\ipykernel_27928\2811396120.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd_feat = ddf.map_partitions(lambda df: df.groupby("ticker").apply(add_features))
C:\Users\Victor Lopez\AppData\Local\Temp\ipykernel_27928\2811396120.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd_feat = ddf.map_partitions(lambda df: df.groupby("ticker").apply(add_features))
C:\U

Date   Open   High    Low  Close  Adj Close      Volume  \
ticker                                                                        
ACN    137315 2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0   
       137316 2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0   
       137317 2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0   
       137318 2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0   
       137319 2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0   
       137320 2001-07-26  14.95  14.99  14.50  14.50  10.900705   6335300.0   
       137321 2001-07-27  14.51  14.59  14.50  14.51  10.908223   3524000.0   
       137322 2001-07-30  14.50  14.78  14.50  14.70  11.051059   3654300.0   
       137323 2001-07-31  14.71  15.01  14.60  14.96  11.246520   1429000.0   
       137324 2001-08-01  15.00  15.50  14.90  15.50  11.652478   2087900.0   

                source ticker  Year  Close_lag_1  Adj_Close_lag_1   Returns  \
ticker                                                                        
ACN    137315  ACN.csv    ACN  2001          NaN              NaN       NaN   
       137316  ACN.csv    ACN  2001        15.17        11.404394 -0.010547   
       137317  ACN.csv    ACN  2001        15.01        11.284108 -0.000666   
       137318  ACN.csv    ACN  2001        15.00        11.276587 -0.009333   
       137319  ACN.csv    ACN  2001        14.86        11.171341  0.006057   
       137320  ACN.csv    ACN  2001        14.95        11.238999 -0.030100   
       137321  ACN.csv    ACN  2001        14.50        10.900705  0.000690   
       137322  ACN.csv    ACN  2001        14.51        10.908223  0.013094   
       137323  ACN.csv    ACN  2001        14.70        11.051059  0.017687   
       137324  ACN.csv    ACN  2001        14.96        11.246520  0.036096   

               Hi_lo_range  
ticker                      
ACN    137315         0.29  
       137316         0.25  
       137317         0.46  
       137318         0.27  
       137319         0.30  
       137320         0.49  
       137321         0.09  
       137322         0.28  
       137323         0.41  
       137324         0.60

C:\Users\Victor Lopez\AppData\Local\Temp\ipykernel_27928\2811396120.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd_feat = ddf.map_partitions(lambda df: df.groupby("ticker").apply(add_features))
C:\Users\Victor Lopez\AppData\Local\Temp\ipykernel_27928\2811396120.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd_feat = ddf.map_partitions(lambda df: df.groupby("ticker").apply(add_features))
C:\U

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Convert Dask Dataframe to Pandas Data Frame
pandas_dd = dd_feat.compute() 

# Drop the hierarchical index to avoid conflicts
pandas_dd = pandas_dd.reset_index(drop=True) 

# Sort the data by ticker and date to ensure correct rolling window calculations
pandas_dd = pandas_dd.sort_values(["ticker","Date"])

# Add a new column with the 10-day moving average of the daily returns
pandas_dd["Returns_W10days"] = pandas_dd.groupby("ticker")["Returns"].transform(
    lambda x: x.rolling(10).mean()
)

pandas_dd.head(15) # Display the first 15 rows for verification

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_lo_range,Returns_W10days
0,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,NaN,NaN,NaN,0.29,NaN
1,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,15.17,11.404394,-0.010547,0.25,NaN
2,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,15.01,11.284108,-0.000666,0.46,NaN
3,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,15.00,11.276587,-0.009333,0.27,NaN
4,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,14.86,11.171341,0.006057,0.30,NaN
5,2001-07-26,14.95,14.99,14.50,14.50,10.900705,6335300.0,ACN.csv,ACN,2001,14.95,11.238999,-0.030100,0.49,NaN
6,2001-07-27,14.51,14.59,14.50,14.51,10.908223,3524000.0,ACN.csv,ACN,2001,14.50,10.900705,0.000690,0.09,NaN
7,2001-07-30,14.50,14.78,14.50,14.70,11.051059,3654300.0,ACN.csv,ACN,2001,14.51,10.908223,0.013094,0.28,NaN
8,2001-07-31,14.71,15.01,14.60,14.96,11.246520,1429000.0,ACN.csv,ACN,2001,14.70,11.051059,0.017687,0.41,NaN
9,2001-08-01,15.00,15.50,14.90,15.50,11.652478,2087900.0,ACN.csv,ACN,2001,14.96,11.246520,0.036096,0.60,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
> Yes — converting to Pandas was necessary in this case to calculate the 10-day moving average return correctly. While Dask supports .rolling(), it performs the calculation separately within each partition. That means it loses the continuity needed to compute a rolling average over the entire dataset, especially when values span across partition boundaries. By converting to Pandas, we ensured the full context was available, and the rolling window calculation was accurate across all rows.
+ Would it have been better to do it in Dask? Why?
> Only if the dataset was too large for memory. Dask is better for big data, but rolling calculations break across partitions and require more advanced handling. Since our dataset was small and Pandas is easier to work with, it was the better choice here.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.